In [ ]:
from skm_tea.metrics import QuantitativeKneeMRI
from meddlr.data.data_utils import collect_mask
import dosma as dm
from dosma.scan_sequences import QDess

https://dosma.readthedocs.io/en/latest/


arr = oF.categorical_to_one_hot(segmentation.A, channel_dim=-1)
arr = collect_mask(arr, (0, 1, (2,3), (4,5)), out_channel_first=False)
print(arr.shape)
seg_gt = dm.MedicalVolume(arr, affine=segmentation.affine)
print(seg_gt)
echo1 = dm.MedicalVolume(image[:,:,:,0,0],affine=segmentation.affine)
echo2 = dm.MedicalVolume(image[:,:,:,1,0],affine=segmentation.affine)


In [ ]:
metadata: pd.DataFrame = MetadataCatalog.get("skmtea_v1_train").scan_metadata
metadata = metadata[metadata["MTR_ID"] == scan["scan_id"]]



In [ ]:
def get_t1(seg: dm.MedicalVolume):
  """Build T1 maps based on the segmentation.

  `seg[..., 3]` should correspond to the meniscus segmentation map.

  Args:
    seg (dm.MedicalVolume): A one-hot encoded segmentation mask, where the
      last dimension is the channel dimension.

  Returns:
    dm.MedicalVolume: The estimated T1 map (in milliseconds).
  """
  t1 = dm.MedicalVolume(np.ones(seg.shape[:3]) * 1200, seg.affine).to(seg.device)
  t1[seg.A[..., 3].astype(bool)] = 1000
  return t1

t1_gt = get_t1(seg_gt)

In [ ]:
def compute_t2_map(t1: dm.MedicalVolume):
  qdess = QDess([echo1, echo2]).to(t1.device)
  t2map = qdess.generate_t2_map(
      suppress_fat=True,
      suppress_fluid=True,
      gl_area=float(metadata["SpoilerGradientArea"]),
      tg=float(metadata["SpoilerGradientTime"]),
      tr=float(metadata["RepetitionTime"]),
      te=float(metadata["EchoTime1"]),
      alpha=float(metadata["FlipAngle"]),
      t1=t1,
      nan_bounds=(0, 100),
      nan_to_num=True,
  )
  return t2map.volumetric_map

In [ ]:
sl = 80 # Sagittal slice to plot
nrows = 1
hsize = 5
wsize = hsize / kspace.shape[0] * kspace.shape[1]
_, axs = plt.subplots(nrows, 2, figsize=(2 * wsize, nrows * hsize))
plot_images(
  [np.abs(image)[:,:,sl,0,0],t2_gt.A[:,:,sl]],
  titles=["Weigthed Echo1 ", "T2 "],
  cmap=["gray","viridis"], show_cbar=[False,True],axs=axs,fontsize=15,overlay=[seg_colorized[:,:,sl,:],None]
)
plt.savefig('/scratch/tmpaquaij/MasterProject/Figures/T2_prediction.png')

In [ ]:
use_subregions = False
use_cpu = use_subregions  # computing subregions is currently limited to the CPU
tissues = ["pc", "fc", "tc", "men"]

qmri_gt = QuantitativeKneeMRI(channel_names=tissues, subregions=use_subregions, use_cpu=use_cpu)
qmri_pred = QuantitativeKneeMRI(channel_names=tissues, subregions=use_subregions, use_cpu=use_cpu)

# Reset the metrics
qmri_gt.reset()
qmri_pred.reset()

# Compute regional qMRI estimates using ground truth and predicted segmentations.
qmri_gt(ids=[scan["scan_id"]], quantitative_map=[t2_gt], sem_seg=[seg_gt], medial_direction=metadata["MedialDirection"])
qmri_pred(ids=[scan["scan_id"]], quantitative_map=[t2_pred], sem_seg=[seg_pred], medial_direction=metadata["MedialDirection"])

print("Ground Truth Regional T2 Estimates:")
display(qmri_gt.to_pandas())

print("Predicted Regional T2 Estimates:")
display(qmri_pred.to_pandas())